In [2]:
import urllib.request
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
filename = 'cats_and_dogs_filtered.zip'
urllib.request.urlretrieve(url, filename)



('cats_and_dogs_filtered.zip', <http.client.HTTPMessage at 0x26209a55348>)

In [2]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd

In [7]:
# This code block unzips the full Cats-v-Dogs dataset to /tmp
# which will create a tmp/PetImages directory containing subdirectories
# called 'Cat' and 'Dog' (that's how the original researchers structured it)

local_zip = 'cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('')
zip_ref.close()

In [3]:
import os
print(os.listdir())


['.ipynb_checkpoints', 'cats_and_dogs_filtered', 'cats_and_dogs_filtered.zip', 'cats_vs_dog_image_augmentation.ipynb']


In [5]:
base_dir = 'cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat/dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')



In [25]:
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )
print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

['.ipynb_checkpoints', 'cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.101.jpg', 'cat.102.jpg', 'cat.103.jpg', 'cat.104.jpg', 'cat.105.jpg']
['.ipynb_checkpoints', 'dog.0.jpg', 'dog.1.jpg', 'dog.10.jpg', 'dog.100.jpg', 'dog.101.jpg', 'dog.102.jpg', 'dog.103.jpg', 'dog.104.jpg', 'dog.105.jpg']


In [7]:
print('total training cat images :', len(os.listdir(train_cats_dir ) ))
print('total training dog images :', len(os.listdir(train_dogs_dir ) ))

print('total validation cat images :', len(os.listdir( validation_cats_dir ) ))
print('total validation dog images :', len(os.listdir( validation_dogs_dir ) ))

total training cat images : 1001
total training dog images : 1001
total validation cat images : 500
total validation dog images : 500


In [26]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])


In [35]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=10,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=10,
        class_mode='binary')

history = model.fit(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=50,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

Found 2014 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/50
100/100 - 25s - loss: 0.8102 - accuracy: 0.6871 - val_loss: 0.5609 - val_accuracy: 0.7040
Epoch 2/50
100/100 - 26s - loss: 0.6000 - accuracy: 0.6820 - val_loss: 0.5149 - val_accuracy: 0.7420
Epoch 3/50
100/100 - 24s - loss: 0.5736 - accuracy: 0.7040 - val_loss: 0.4985 - val_accuracy: 0.7400
Epoch 4/50
100/100 - 26s - loss: 0.5480 - accuracy: 0.7364 - val_loss: 0.4749 - val_accuracy: 0.7800
Epoch 5/50
100/100 - 29s - loss: 0.5543 - accuracy: 0.7294 - val_loss: 0.4989 - val_accuracy: 0.7560
Epoch 6/50
100/100 - 27s - loss: 0.5455 - accuracy: 0.7190 - val_loss: 0.5233 - val_accuracy: 0.7540
Epoch 7/50
100/100 - 26s - loss: 0.5584 - accuracy: 0.7012 - val_loss: 0.4732 - val_accuracy: 0.7620
Epoch 8/50
100/100 - 27s - loss: 0.5078 - accuracy: 0.7380 - val_loss: 0.4556 - val_accuracy: 0.7860
Epoch 9/50
100/100 - 27s - loss: 0.5427 - accuracy: 0.7080 - val_loss: 0.4640 - val_accuracy: 0.7760
Epoch 1

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()